# Анализ NPS клиентов телекоммуникационной компании

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. В нашем распоряжении база данных с результатами опроса клиентов удовлетворенности сервисом NPS.

**Цель исследования:**
Существует гипотеза, что есть факторы, влияющие на рынок NPS. Проверим ее. Также нам следует оценить, насколько успешна политика компании среди клиентов.

**Ход исследования**

Данные получим из базы данных.

На первом этапе требуется выгрузить их и проверить качество. Затем перейдем к проверке гипотезы.

In [68]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [69]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [70]:
query = """ 
SELECT u.user_id,
        u.lt_day,
        CASE
            WHEN u.lt_day < 365 THEN 'новый'
            WHEN u.lt_day >= 365 THEN 'постоянный'
            ELSE 'другие'
        END AS is_new,
        u.age,
        CASE
            WHEN u.gender_segment = 1 THEN 'женщина'
            WHEN u.gender_segment = 0 THEN 'мужчина'
            ELSE 'другие'
        END AS gender,
        u.os_name,
        u.cpe_type_name,
        loc.country,
        loc.city,
        age.title AS title_age_segment,
        traf.title AS title_traf_segment,
        life.title AS title_life_segment,
        u.nps_score,
        CASE
            WHEN u.nps_score > 8 THEN 'сторонники'
            WHEN u.nps_score > 6 AND u.nps_score <= 8 THEN 'нейтралы'
            WHEN u.nps_score <= 6  THEN 'критики'
            ELSE 'другие' 
        END AS nps_group
FROM user AS u
INNER JOIN location AS loc ON u.location_id = loc.location_id
INNER JOIN age_segment AS age ON age.age_gr_id = u.age_gr_id
INNER JOIN traffic_segment AS traf ON traf.tr_gr_id = u.tr_gr_id
INNER JOIN lifetime_segment AS life ON life.lt_gr_id = u.lt_gr_id;
"""

In [78]:
df.head(5)

,user_id,lt_day,is_new,age,gender,os_name,cpe_type_name,country,city,title_age_segment,title_traf_segment,title_life_segment,nps_score,nps_group
0,A001A2,2320,постоянный,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,постоянный,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,постоянный,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники
3,A004TB,4190,постоянный,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонники
4,A004XT,1163,постоянный,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонники


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             502493 non-null  object 
 1   lt_day              502493 non-null  int64  
 2   is_new              502493 non-null  object 
 3   age                 501939 non-null  float64
 4   gender              502493 non-null  object 
 5   os_name             502493 non-null  object 
 6   cpe_type_name       502493 non-null  object 
 7   country             502493 non-null  object 
 8   city                502493 non-null  object 
 9   title_age_segment   502493 non-null  object 
 10  title_traf_segment  502493 non-null  object 
 11  title_life_segment  502493 non-null  object 
 12  nps_score           502493 non-null  int64  
 13  nps_group           502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [73]:
df.isna().sum()

user_id                 0
lt_day                  0
is_new                  0
age                   554
gender                  0
os_name                 0
cpe_type_name           0
country                 0
city                    0
title_age_segment       0
title_traf_segment      0
title_life_segment      0
nps_score               0
nps_group               0
dtype: int64

In [74]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501939 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             501939 non-null  object 
 1   lt_day              501939 non-null  int64  
 2   is_new              501939 non-null  object 
 3   age                 501939 non-null  float64
 4   gender              501939 non-null  object 
 5   os_name             501939 non-null  object 
 6   cpe_type_name       501939 non-null  object 
 7   country             501939 non-null  object 
 8   city                501939 non-null  object 
 9   title_age_segment   501939 non-null  object 
 10  title_traf_segment  501939 non-null  object 
 11  title_life_segment  501939 non-null  object 
 12  nps_score           501939 non-null  int64  
 13  nps_group           501939 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 57.4+ MB


In [75]:
df.duplicated().sum()

0

In [76]:
df.reset_index(drop=True , inplace=True)

In [77]:
df.to_csv('telecomm_csi_traffic.csv')

Презентация проекта: https://public.tableau.com/app/profile/vasilisa.doronina/viz/Book3_17091427338140/Story1?publish=yes